<a href="https://colab.research.google.com/github/shuwang127/636Project/blob/main/Classifers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np

_COLAB_ = 0 if (os.getenv('COLAB_GPU', 'NONE') == 'NONE') else 1
rootPath = './drive/My Drive/Colab Notebooks/' if (_COLAB_) else './'

In [2]:
# read data.
data = pd.read_csv(rootPath + 'training_data.csv')

In [3]:
# fill missing features.
data = data.replace(' ?', np.nan)
for index, row in data.iteritems():
    data[index].fillna(data[index].mode()[0], inplace=True)
data.head()

,ID (this is not a feature),age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,1,36,Private,355053,HS-grad,9,Separated,Other-service,Unmarried,Black,Female,0,0,28,United-States,<=50K
1,2,30,Self-emp-inc,132601,Bachelors,13,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,>50K
2,3,19,Private,63814,Some-college,10,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,18,United-States,<=50K
3,4,44,Private,112507,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,<=50K
4,5,51,Self-emp-inc,126850,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,65,United-States,<=50K


In [4]:
# get labels.
Y = data['salary'].replace([' <=50K' , ' >50K'], [0, 1])
Y.head()

0    0
1    1
2    0
3    0
4    0
Name: salary, dtype: int64

In [5]:
# get features.
X = data.drop(['ID (this is not a feature)', 'salary'], axis=1)
# normalize numeric features.
from sklearn.preprocessing import StandardScaler
numcol = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
scaler = StandardScaler()
X[numcol] = scaler.fit_transform(X[numcol])
# get one-hot vectors.
X = pd.get_dummies(X)
X.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital-status_ Divorced,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ Adm-clerical,occupation_ Armed-Forces,occupation_ Craft-repair,...,native-country_ Canada,native-country_ China,native-country_ Columbia,native-country_ Cuba,native-country_ Dominican-Republic,native-country_ Ecuador,native-country_ El-Salvador,native-country_ England,native-country_ France,native-country_ Germany,native-country_ Greece,native-country_ Guatemala,native-country_ Haiti,native-country_ Holand-Netherlands,native-country_ Honduras,native-country_ Hong,native-country_ Hungary,native-country_ India,native-country_ Iran,native-country_ Ireland,native-country_ Italy,native-country_ Jamaica,native-country_ Japan,native-country_ Laos,native-country_ Mexico,native-country_ Nicaragua,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Peru,native-country_ Philippines,native-country_ Poland,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,-0.193656,1.560810,-0.420024,-0.145196,-0.218481,-1.002552,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,-0.630858,-0.538524,1.136932,-0.145196,-0.218481,-0.033634,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,-1.432394,-1.187683,-0.030785,-0.145196,-0.218481,-1.809983,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0.389280,-0.728156,-0.030785,-0.145196,-0.218481,-0.033634,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0.899348,-0.592797,-0.420024,-0.145196,-0.218481,1.984944,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [6]:
# separate training data and validation data.
from sklearn.model_selection import train_test_split
XTrain, XTest, YTrain, YTest = train_test_split(X, Y, test_size=0.2)
print("size of training data is", XTrain.shape)
print("size of testing data is", XTest.shape)

size of training data is (31073, 105)
size of testing data is (7769, 105)


In [7]:
# evaluation.
from sklearn.metrics import accuracy_score, confusion_matrix
def Evaluation(YTest, YPred, method=''):
    print('====================== ' + method + ' ======================')
    acc = accuracy_score(YTest, YPred) * 100
    print('Accuracy is %.3f%%.' % acc)
    conf = confusion_matrix(YTest, YPred)
    print(pd.DataFrame(conf, columns=['Pred-Neg','Pred-Pos'], index=['Actl-Neg', 'Actl-Pos']))
    precision = conf[1][1] / (conf[0][1] + conf[1][1]) if (conf[0][1] + conf[1][1]) else 0
    recall = conf[1][1] / (conf[1][0] + conf[1][1]) if (conf[1][0] + conf[1][1]) else 0
    F1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
    print('Precision is %.3f.' % precision)
    print('Recall is %.3f.' % recall)
    print('F1 score is %.3f.' % F1)

In [8]:
# NaiveBayes.
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
Evaluation(YTest, YPred, 'Naive Bayes')

====================== Naive Bayes ======================
Accuracy is 61.308%.
          Pred-Neg  Pred-Pos
Actl-Neg      3010      2873
Actl-Pos       133      1753
Precision is 0.379.
Recall is 0.929.
F1 score is 0.538.


In [9]:
# Least Square Regression.
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
YPred = (YPred > 0.5)
Evaluation(YTest, YPred, 'Least Square Regression')

====================== Least Square Regression ======================
Accuracy is 84.078%.
          Pred-Neg  Pred-Pos
Actl-Neg      5572       311
Actl-Pos       926       960
Precision is 0.755.
Recall is 0.509.
F1 score is 0.608.


In [10]:
# Ridge Regression.
from sklearn.linear_model import Ridge
model = Ridge(alpha=1e-2, normalize=True)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
YPred = (YPred > 0.5)
Evaluation(YTest, YPred, 'Ridge Regression')

====================== Ridge Regression ======================
Accuracy is 84.078%.
          Pred-Neg  Pred-Pos
Actl-Neg      5575       308
Actl-Pos       929       957
Precision is 0.757.
Recall is 0.507.
F1 score is 0.607.


In [11]:
# Lasso Regression.
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
model = Lasso(alpha=1e-6)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
YPred = (YPred > 0.5)
Evaluation(YTest, YPred, 'Lasso Regression')

====================== Lasso Regression ======================
Accuracy is 84.078%.
          Pred-Neg  Pred-Pos
Actl-Neg      5572       311
Actl-Pos       926       960
Precision is 0.755.
Recall is 0.509.
F1 score is 0.608.


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1534.2048730549477, tolerance: 0.5658594921636153
  positive)


In [12]:
# Logistic Regression.
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
Evaluation(YTest, YPred, 'Logistic Regression')

====================== Logistic Regression ======================
Accuracy is 85.301%.
          Pred-Neg  Pred-Pos
Actl-Neg      5504       379
Actl-Pos       763      1123
Precision is 0.748.
Recall is 0.595.
F1 score is 0.663.


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [13]:
# K-means.
from sklearn.cluster import KMeans
model = KMeans(n_clusters=2, random_state=0)
model.fit(X)
YPred = model.labels_
import scipy.stats as stats
if (stats.pearsonr(Y, YPred)[0] < 0):
    YPred = (YPred == 0)
Evaluation(Y, YPred, 'K-means')

====================== K-means ======================
Accuracy is 66.951%.
          Pred-Neg  Pred-Pos
Actl-Neg     17521     11995
Actl-Pos       842      8484
Precision is 0.414.
Recall is 0.910.
F1 score is 0.569.


In [14]:
# Decision Tree.
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(criterion="gini", splitter="best", max_features=None, min_samples_split=300, min_samples_leaf=1, random_state=0)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
Evaluation(YTest, YPred, 'Decision Tree')

====================== Decision Tree ======================
Accuracy is 86.008%.
          Pred-Neg  Pred-Pos
Actl-Neg      5582       301
Actl-Pos       786      1100
Precision is 0.785.
Recall is 0.583.
F1 score is 0.669.


In [15]:
# Random Forest.
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=100, criterion='gini', max_features = 'sqrt', n_jobs=-1, verbose=1, random_state=0)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
Evaluation(YTest, YPred, 'Random Forest')

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s


====================== Random Forest ======================
Accuracy is 85.082%.
          Pred-Neg  Pred-Pos
Actl-Neg      5460       423
Actl-Pos       736      1150
Precision is 0.731.
Recall is 0.610.
F1 score is 0.665.


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.2s finished


In [16]:
# Support Vector Machine.
from sklearn.svm import SVC
model = SVC(kernel='linear', C=1)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
Evaluation(YTest, YPred, 'Support Vector Machine')

====================== Support Vector Machine ======================
Accuracy is 84.464%.
          Pred-Neg  Pred-Pos
Actl-Neg      5522       361
Actl-Pos       846      1040
Precision is 0.742.
Recall is 0.551.
F1 score is 0.633.


In [17]:
# Neural Network.
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(32,8), activation='relu', solver='lbfgs', max_iter=100)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
Evaluation(YTest, YPred, 'Neural Network')

====================== Neural Network ======================
Accuracy is 85.455%.
          Pred-Neg  Pred-Pos
Actl-Neg      5452       431
Actl-Pos       699      1187
Precision is 0.734.
Recall is 0.629.
F1 score is 0.678.


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
